# **Ejercicios de pair programming Módulo 3 Sprint 1**
## **Regresión logística: Lección 6**
### **Random Forest**

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

Hasta ahora hemos ajustado el modelo usando una Regresión Logística, pero como hemos aprendido, podemos usar el Random Forest en este tipo de problemas. Los objetivos de este pair programming :

 - Ajustad un modelo de Random Forest a nuestros datos.


In [2]:
df = pd.read_csv("../datos/Invistico_Airline_procesado.csv", index_col = 0)
df.head()

,age,flight_distance,departure_delay_in_minutes,gender_oe,customer_type_oe,type_of_travel_oe,class_oe,seat_comfort_oe,departure_arrival_time_convenient_oe,food_and_drink_oe,...,inflight_entertainment_oe,online_support_oe,ease_of_online_booking_oe,on_board_service_oe,leg_room_service_oe,baggage_handling_oe,checkin_service_oe,cleanliness_oe,online_boarding_oe,satisfaction
0,0.236257,0.491272,-0.386481,0,0,0,0,2,4,4,...,2,3,2,2,2,1,2,2,2,0
1,-0.359008,0.297525,-0.281414,1,0,1,1,2,3,2,...,2,5,5,3,2,3,4,3,5,0
2,-0.557430,2.030540,8.176472,0,0,0,1,4,3,3,...,4,4,2,3,3,1,4,1,4,1
3,1.889773,0.487377,-0.386481,0,0,0,1,1,4,2,...,4,4,1,1,1,0,1,1,2,0
4,-0.689712,1.161111,-0.071280,0,0,0,0,3,2,2,...,3,3,3,3,5,3,3,4,3,0


In [3]:
X = df.drop("satisfaction", axis = 1)
y = df['satisfaction']

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

bosque = RandomForestClassifier(max_depth=25, max_features=5, min_samples_leaf=10,
                       min_samples_split=10, random_state=42)
 
bosque.fit(x_train, y_train)

y_pred_test = bosque.predict(x_test)
y_pred_train = bosque.predict(x_train)

 - Calculad las métricas a nuestro nuevo modelo.


In [5]:
def metricas(clases_reales_test, clases_predichas_test, clases_reales_train, clases_predichas_train, modelo):
    
    # para el test
    accuracy_test = accuracy_score(clases_reales_test, clases_predichas_test)
    precision_test = precision_score(clases_reales_test, clases_predichas_test)
    recall_test = recall_score(clases_reales_test, clases_predichas_test)
    f1_test = f1_score(clases_reales_test, clases_predichas_test)
    kappa_test = cohen_kappa_score(clases_reales_test, clases_predichas_test)

    # para el train
    accuracy_train = accuracy_score(clases_reales_train, clases_predichas_train)
    precision_train = precision_score(clases_reales_train, clases_predichas_train)
    recall_train = recall_score(clases_reales_train, clases_predichas_train)
    f1_train = f1_score(clases_reales_train, clases_predichas_train)
    kappa_train = cohen_kappa_score(clases_reales_train, clases_predichas_train)
    

    
    df = pd.DataFrame({"accuracy": [accuracy_test, accuracy_train], 
                       "precision": [precision_test, precision_train],
                       "recall": [recall_test, recall_train], 
                       "f1": [f1_test, f1_train],
                       "kapppa": [kappa_test, kappa_train],
                       "set": ["test", "train"]})
    
    df["modelo"] = modelo
    return df

In [6]:
rf_results = metricas(y_test, y_pred_test, y_train, y_pred_train, "Random Forest")
rf_results

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.959986,0.969476,0.949915,0.959596,0.919972,test,Random Forest
1,0.970782,0.980884,0.960272,0.970468,0.941564,train,Random Forest


 - Comparad las métricas con el modelo hecho hasta ahora. ¿Cuál es mejor?

In [7]:
resultados_metricas = pd.read_csv("../datos/Invistico_Airline_metricas.csv", index_col = 0)
resultados_metricas.head()

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.844677,0.848542,0.839300,0.843896,0.689355,test,Regresión logistica
1,0.842790,0.848524,0.834521,0.841464,0.685579,train,Regresión logistica
0,0.933860,0.945545,0.920811,0.933014,0.867722,test,Decision Tree gs
1,0.950730,0.962340,0.938163,0.950098,0.901460,train,Decision Tree gs


In [8]:
# unimos el df con los resultados del modelo de regresión logística

resultados_metricas = pd.concat([resultados_metricas, rf_results], axis = 0)
resultados_metricas

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.844677,0.848542,0.839300,0.843896,0.689355,test,Regresión logistica
1,0.842790,0.848524,0.834521,0.841464,0.685579,train,Regresión logistica
0,0.933860,0.945545,0.920811,0.933014,0.867722,test,Decision Tree gs
1,0.950730,0.962340,0.938163,0.950098,0.901460,train,Decision Tree gs
0,0.959986,0.969476,0.949915,0.959596,0.919972,test,Random Forest
1,0.970782,0.980884,0.960272,0.970468,0.941564,train,Random Forest


`Como podemos comprobar, el modelo Random Forest es el que mejores resultados nos ha dado. Consideramos que todas las métricas son bastante buenas, y que el modelo es robusto y fiable en sus predicciones. El kappa está por encima de 0.9 tanto en el train como en el test, y el resto de las métricas en general son muy altas y homogéneas.`